In [2]:
from transformers import pipeline
import torch
print(torch.cuda.is_available())
print(torch.__version__)

True
2.5.1+cu124


In [2]:
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device=0)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Lưu Model và Tokenizer

In [3]:
save_directory = "./xlm-roberta-large-xnli"

# Save the model and tokenizer
classifier.model.save_pretrained(save_directory)
classifier.tokenizer.save_pretrained(save_directory)

('./xlm-roberta-large-xnli\\tokenizer_config.json',
 './xlm-roberta-large-xnli\\special_tokens_map.json',
 './xlm-roberta-large-xnli\\sentencepiece.bpe.model',
 './xlm-roberta-large-xnli\\added_tokens.json',
 './xlm-roberta-large-xnli\\tokenizer.json')

### Load Model và tokenizer

In [4]:
import time
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# Measure time to load model and tokenizer
start_time = time.time()

# Load the model and tokenizer from the saved directory
model = AutoModelForSequenceClassification.from_pretrained('./xlm-roberta-large-xnli')
tokenizer = AutoTokenizer.from_pretrained('./xlm-roberta-large-xnli')
load_time = time.time() - start_time
print(f"Time taken to load model and tokenizer: {load_time:.4f} seconds")

Time taken to load model and tokenizer: 3.2712 seconds


### Prompt và dáp án check

In [5]:
prompt = [
    "Hello mọi người tôi là Minh chủ chuyến đi này",
    "Mọi người đều biết là đi Đà Nẵng rồi nhỉ",
    "Chốt lịch là thứ 7 đến thứ 2 nhé mọi người.",
    "Hmm nghe có vẻ đưuọc đó",
    "Sáng thứ 7 7h tập trung ở sân bay Tân Sơn Nhất nhé, đi chuyến sớm.",
    "Sao sơm thế ?",
    "Vì chuyến bay sớm, nên phải tập trung sớm để check-in.",
    "Haiz lại phải dậy sớm rồi 😢",
    "Không sao, chuyến đi sẽ thú vị lắm đó.",
    "Còn lịch trình cụ thể thì mai bàn nhé",
    "Không được",
    "Bàn luôn đi mai tôi phải học rồi",
    "Bà này lắm chuyện ghê",
    "Đấm nhau không",
    "Đme đánh luôn này set cái keofof",
    "Mày tuổi gì",
    "Thôi thôi tôi xin lại chiến tranh tg t3 bây h",
    "Trưa tới Đà Nẵng đi ăn bánh tráng cuốn thịt heo nha, nghe bảo quán Trần ngon lắm.",
    "À ừ tôi cũng nghe nói về quán đó rồi, chắc phải thử thử hehehe =)).",
    "Chiều thì đi Ngũ Hành Sơn nhé, check-in sống ảo các kiểu.",
    "Buổi tối làm một vòng ra cầu Rồng, tiện ngắm cầu phun lửa.",
    "thôi cái này để tối hôm sau đi mới đến mệt mà",
    "Được, tối cn đi cầu Rồng vậy.",
    "Ủa vậy chiều thứ 2 về hay tối về?",
    "Chiều thứ 2 về luôn nhé, tối về trễ lắm.",
    "Ok, vậy lịch chốt là từ thứ 7 sáng đến chiều thứ 2 nhé.",
    "Mọi người đã xem phim mới ra chưa, nghe nói hay lắm.",
    "Dạo này thời tiết Đà Nẵng nắng nóng dễ chịu ghê.",
    "Tối qua tôi mới đọc một cuốn sách thú vị, để khi nào kể cho mọi người nghe 🥸",
    "Anh em có ai thích mua đồ lưu niệm không, nhiều đồ đẹp lắm.",
    "Lần này đi mà mệt quá, chắc về phải nghỉ thêm vài ngày.",
    "Chị tôi ở Đà Nẵng bảo lần sau tới thì ghé qua nhà chơi :)))",
    "Có ai biết Đà Nẵng có chỗ nào đổi ngoại tệ không nhỉ?",
    "Nhớ đem theo máy ảnh để chụp choẹt ở Ngũ Hành Sơn nha!",
    "Tối mai chúng ta ăn gì nhỉ? Có món gì đặc sản không?",
    "Anh em nhớ giữ vé máy bay cẩn thận nha, đừng quên khi lên máy bay!",
    "Ai đặt xe từ sân bay về khách sạn chưa nhỉ?",
    "Sáng chủ nhật đi Bà Nà luôn không mọi người?",
    "Sáng thứ 2 mọi người ăn sáng nhanh rồi đi Hội An nhé.",
    "Chơi game không",
    "Nghe nói dạo này ai đó thích lắm😞",
    "😞",
    "Dm ai đây",
    "Nyc@Nguyễn Minh à",
    "BROS FAMILY JUST KNOW HE BREAK NNN STREAK",
    "???",
    "ăn bún đi",
    "trông lạ vl",
    "giờ người ta khác rồi anh",
    "mấy chục năm chả gặp lại k lạ",
    "Ăn nói :)))",
    "4 năm chứ có ít gì iem",
    "??? a",
    "mẹ dậy thị hết rồi chứ có phải con nít đ đâu mà khác trừ phi pttm",
    "?:)",
    "7h30",
    "thế k tính gầy đi béo lên à",
    "Xi mà",
    "béo lên cũng chả khác mẹ gì",
    "Hnay đi ăn ko mn? Đói quáaa 🤤🍕",
    "Có ai rủ đi bơi ko, nóng chảy mỡ rồi 🥵",
    "mà chỗ ở ở đâu thế",
    "Chỗ ở gần biển, đi bộ ra biển mất 5p thôi",
    "Ái dà vip pro",
    "Thế lại tốn tiền rồi :(",
    "Không sao =))) có gì tôi bù"
    "VCL đội ơn ông chủ"
    "=))))",
    "Tối nay 8h off ở quán cũ nha 🕗",
    "Ko ra đc đâu, bận sml 😤",
    "dme =)))",
    "trông dài mà dược 5 câu :((",
    "cứ nahwsn bình thg đi",
    "để tôi cop hehehe",
    "mai mấy giờ lên gặp thầy Trình ? ",
    "huhu tôi đẹp trai quá phải làm sao đây, ai đó cứu tôi với, mỗi lần soi gương tôi không chịu nổi 🙁",
    "ông là Gojou Satoru nên ông mạnh nhất ? Hay vì ông mạnh nhất nên ông là Gojou Satoru ?",
    "xin lỗi, Amanai, Hiện tại không phải tôi đang tức giận vì cô hay gì đâu.",
    "tôi đã làm hỏng hết rồi, anh không có lỗi gì đâu, Gettou",
    "Thứ 7 này đi chơi chỗ nào vui vui đi mn, chán ở nhà rồi 😒",
    "Lol trễ hẹn nữa là thôi luôn nha 🚫",
    "Yêu chưa kịp mà bị phũ rồi 😩💔",
    "Đang kẹt xe sml luôn, chắc 10p nữa tới 🤦‍♂️",
    "Chó dũng nhớ mang quần bơi =))",
    "À để tôi "
    "Đi Đà Lạt chuyến này, ai chưa đặt phòng thì lo lẹ đi nha 🏞️",
    "Sáng mai 5h tập trung ở cổng trường nhé! ⏰",
    "Nhìn nắng thế này đi biển là hết ý luôn 😍🌊",
    "Đường xa vl, đi kiểu này chắc mệt phờ râu 😅",
    "Ai làm tài xế cho tui uống cà phê nào? ☕😂",
    "Cuối cùng là từ Đà Nẵng, về thẳng sân bay luôn phải không?",
    "À tối hôm thứ 2 về ae làm kèo nhậu không",
    "Triển luôn bạn êi",
    "Để tôi mang camera đi cho",
    "Để tôi cầm camera đi cho",
    "Mang theo kem đánh răn cho chắc",
    "Chuẩn bị thuốc men đi nhé",
    "Ai cần mang thuốc say xe không",
    "Đi",
    "Đi",
    "+1",
    "Coi bộ cả đám lười di chuyển rồi nha, đi bộ xíu chắc tạch hết 😂",
    "Chiều nay tầm 4h tập trung tại quán nước quen nha! 🥤",
    "Sáng hôm sau mn dậy lúc 7h đc ko? Để đi thác sớm hơn 🌄",
    "Muốn chill thì đến đúng giờ nha, ko là thôi á 😂",
    "Đi cùng đứa nào mà thích rề rà là tui mệt đấy nha 😩",
    "Mày đặt phòng r đấy à? Định bùng hả? 🙄",
    "Sáng nay cắm trại ở đây, nhậu nhẹt tối nay nè 🍻⛺"
]
# export this variable

human_check = [0, 1, 2, 4, 6, 17, 19, 20, 21, 22, 23, 24,
               25, 31, 33, 37, 38, 55, 66, 77, 82, 86, 87, 93, 94, 98]

human_check_todolist  = [
    33, 35, 81,  90,92,93
]

### Filter emoji độ dài chữ số lượng từ

In [6]:
import re

def is_meaningless_message(message, min_length=8, min_word_count=3):
    message = message.strip()
    
    time_pattern = r'^\d{1,2}(?:h\d{1,2})?$|^\d{1,2}\s?giờ(?:\s?[sáng|chiều|tối]?)?$'
    if re.match(time_pattern, message.lower()):
        return False  # Time-only messages are considered meaningful
    
    date_pattern = r'^(?:\d{1,2}\/\d{1,2}\/\d{2,4}|(?:thứ|ngày)\s?\d{1,2}(?:\s?(?:tháng)?\s?\d{1,2})?(?:\s?(?:năm)?\s?\d{2,4})?)$'
    if re.match(date_pattern, message.lower()):
        return False
    
    words = re.findall(r'\b\w+\b', message)
    if len(words) < min_word_count:
        return True

    if len(message) < min_length:
        return True
    
    if not re.search(r'\w', message):
        return True
    
    return False

### Chạy model phân loại đoạn chat

In [7]:
# Test with the example input
def classify_sentence(prompt_input):
    count_not_relate = 0
    count_filtered = 0
    candidate_labels = ["travel schedule" ]
    modified_labels = [f"{label}" for label in candidate_labels]
    modified_labels.append("not travel schedule")
    dif_senc = []
    related_messages = []
    print("Tổng hợp Tất cả câu phân loại")

    for i,text in enumerate(prompt_input):
        if is_meaningless_message(text):
            count_filtered += 1
            continue
        sequence_to_classify = text
        output = classifier(sequence_to_classify, modified_labels, multi_label=False,batch_size=16)
    
        if output["labels"][0] == "not travel schedule":
            count_not_relate += 1
            print(f'{i} {text} ❌ {"✅" if i in human_check else "❌"} ' + str(output["scores"]))
        else:
            related_messages.append(f'{text}')
            print(f'{i} {text} ✅ {"✅" if i in human_check else "❌"} '+  str(output["scores"]))
        if i in human_check:
            if output["labels"][0] == "not travel schedule":
                dif_senc.append(f'{i} {text} ❌ ✅' + str(output["scores"]))
        else:
            if output["labels"][0] != "not travel schedule":
                dif_senc.append(f'{i} {text} ✅ ❌' + str(output["scores"]))
    
        # print(f"Time taken to classify sequence: {execution_time:.2f} seconds")

    print("NOt relate to travel: ",count_not_relate)
    print("Filtered: ",count_filtered, " / ", len(prompt_input)) 
    print("")
    print("Tổng hợp các câu lệch đáp án",len(dif_senc))
    for x in dif_senc:
        print(x)
    print("")
    print("Tổng hợp các câu liên quan đến du lịch " ,len(related_messages))
    for x in related_messages:
        print(x)
    return related_messages

related_messages = classify_sentence(prompt)
    

Tổng hợp Tất cả câu phân loại
0 Hello mọi người tôi là Minh chủ chuyến đi này ✅ ✅ [0.8469131588935852, 0.1530868262052536]
1 Mọi người đều biết là đi Đà Nẵng rồi nhỉ ✅ ✅ [0.7232663631439209, 0.2767335772514343]
2 Chốt lịch là thứ 7 đến thứ 2 nhé mọi người. ✅ ✅ [0.9778636693954468, 0.022136308252811432]
3 Hmm nghe có vẻ đưuọc đó ❌ ❌ [0.7165447473526001, 0.2834553122520447]
4 Sáng thứ 7 7h tập trung ở sân bay Tân Sơn Nhất nhé, đi chuyến sớm. ✅ ✅ [0.9611651301383972, 0.038834843784570694]


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


5 Sao sơm thế ? ❌ ❌ [0.8485667705535889, 0.15143321454524994]
6 Vì chuyến bay sớm, nên phải tập trung sớm để check-in. ✅ ✅ [0.9851973652839661, 0.014802614226937294]
7 Haiz lại phải dậy sớm rồi 😢 ❌ ❌ [0.6140130162239075, 0.38598698377609253]
8 Không sao, chuyến đi sẽ thú vị lắm đó. ✅ ❌ [0.7243807315826416, 0.2756192982196808]
9 Còn lịch trình cụ thể thì mai bàn nhé ❌ ❌ [0.9918280839920044, 0.008171928115189075]
11 Bàn luôn đi mai tôi phải học rồi ✅ ❌ [0.5469433069229126, 0.4530566930770874]
12 Bà này lắm chuyện ghê ❌ ❌ [0.8455005288124084, 0.15449945628643036]
13 Đấm nhau không ❌ ❌ [0.9540022611618042, 0.0459977425634861]
14 Đme đánh luôn này set cái keofof ❌ ❌ [0.812101423740387, 0.18789856135845184]
15 Mày tuổi gì ❌ ❌ [0.9239166378974915, 0.07608333230018616]
16 Thôi thôi tôi xin lại chiến tranh tg t3 bây h ❌ ❌ [0.6747419238090515, 0.3252580761909485]
17 Trưa tới Đà Nẵng đi ăn bánh tráng cuốn thịt heo nha, nghe bảo quán Trần ngon lắm. ✅ ✅ [0.917151689529419, 0.08284835517406464]
18 À

### Fetch api gemini để đưa ra json lịch trình

In [8]:
initialPromptSchedule = '''
Hãy giúp tôi tóm tắt các đoạn hội thoại về chuyến đi và tạo lịch trình dưới dạng JSON string bỏ cái ngoặc ```json``` không cần xuống dòng với dữ liệu đầu vào có đoạn hội thoại, các ngày đi, địa điểm đi.Áp dụng các thời gian mặc định sau:
"sáng": 8h00
"chiều": 15h00
"tối": 19h00

Input: 
Conversation:
1. "Chốt đi sáng thứ 6 đến tối chủ nhật nhé."
2. "Sáng 8h hôm đầu đi ăn bún bò ở quán Bún Thanh Hoa nhé."
3. "Chiều đi tham quan Hạ Long."
4. "Tối về ăn hải sản."
5. "Ơ vậy tập trung ở đâu."
6. "Sáng thứ 6 tập trung ở Sơn Tây, Hà Nội." 
7. "Tối chủ nhật thì không được rồi,chiều về được không"
8. "Được, chiều chủ nhật về."

Travel dates: 25/10/2024, 26/10/2024, 27/10/2024.
Location: Hạ Long.

Output:
{
"timeline":[
{"day":"2024-10-25","events":[{"time":"2024-10-25T08:00:00","name":"Khởi hành đến Hạ Long","place":"Sơn Tây, Hà Nội"},{"time":"2024-10-25T19:00:00","name":"Nghỉ ngơi, ăn tối","place":""}]},
{"day":"2024-10-26","events":[{"time":"2024-10-26T08:00:00","name":"Ăn bún bò","place":"quán Bún Thanh Hoa"},{"time":"2024-10-26T15:00:00","name":"Tham quan","place":""},{"time":"2024-10-26T19:00:00","name":"Ăn hải sản","place":""}]},
{"day":"2024-10-27","events":[{"time":"2024-10-27T15:00:00","name":"Về","place":""}]}
]
}
'''

In [ ]:
import google.generativeai as genai
import json
from dotenv import load_dotenv
import os


load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

# Sample related messages
formatted_messages = '\n'.join(f'{i+1}. "{msg}"' for i, msg in enumerate(related_messages))
data = {
    "prompt": 'Conversation:' + formatted_messages + '''\nTravel dates: 02/11/2024, 03/11/2024, 04/11/2024.\n
                Location: Đà Nẵng.'''
}
chat =  model.start_chat(
    history=[
                {"role": "user","parts":  initialPromptSchedule},
                {"role": "model","parts": "Tôi hiểu rồi, tôi sẽ nhận vào cuộc hội thoại, ngày đi và địa điểm đi tổng hợp lại thành 1 json string có thể chuyển lại thành json có chứa thông tin về lịch trình chuyến đi."}
            ]
  )

response = chat.send_message(data["prompt"])
print(response.text)
parsed_response = json.loads(response.text)
formatted_json = json.dumps(parsed_response, indent=4, ensure_ascii=False)
print(formatted_json)  # Display in a well-formatted way



{
"timeline":[
{"day":"2024-11-02","events":[{"time":"2024-11-02T07:00:00","name":"Tập trung","place":"Sân bay Tân Sơn Nhất"},{"time":"2024-11-02T12:00:00","name":"Ăn bánh tráng cuốn thịt heo","place":"quán Trần"},{"time":"2024-11-02T15:00:00","name":"Tham quan","place":"Ngũ Hành Sơn"},{"time":"2024-11-02T19:00:00","name":"Đi dạo","place":"Cầu Rồng"}]},
{"day":"2024-11-03","events":[{"time":"2024-11-03T08:00:00","name":"Tham quan","place":"Bà Nà"},{"time":"2024-11-03T19:00:00","name":"Đi dạo","place":"Cầu Rồng"}]},
{"day":"2024-11-04","events":[{"time":"2024-11-04T07:30:00","name":"Ăn sáng","place":""},{"time":"2024-11-04T08:00:00","name":"Tham quan","place":"Hội An"},{"time":"2024-11-04T15:00:00","name":"Về","place":""}]}
]
}
{
    "timeline": [
        {
            "day": "2024-11-02",
            "events": [
                {
                    "time": "2024-11-02T07:00:00",
                    "name": "Tập trung",
                    "place": "Sân bay Tân Sơn Nhất"
              

### Classify todo list

In [13]:
# Test with the example input
def classify_sentence_todo(prompt_input):
    count_not_relate = 0
    count_filtered = 0
    candidate_labels = ["to-do list memorize packing" ]
    modified_labels = [f"{label}" for label in candidate_labels]
    modified_labels.append("not to-do list memorize packing")
    dif_senc = []
    related_messages = []
    rank = {
        "to-do list memorize packing":[],
        "not to-do list memorize packing": []
    }
    print("Tổng hợp Tất cả câu phân loại")

    for i,text in enumerate(prompt_input):
        if is_meaningless_message(text):
            count_filtered += 1
            continue
        sequence_to_classify = text
        output = classifier(sequence_to_classify, modified_labels, multi_label=False,batch_size=16)
    
        if output["labels"][0] == "not to-do list memorize packing":
            if(output["scores"][1] > 0.4):
                related_messages.append(f'{text}')
                print(f'{i} {text} ✅ {"✅" if i in human_check_todolist else "❌"} '+  str(output["scores"]))
            else:
                count_not_relate += 1
                print(f'{i} {text} ❌ {"✅" if i in human_check_todolist else "❌"} ' + str(output["scores"]))
        else:
            related_messages.append(f'{text}')
            print(f'{i} {text} ✅ {"✅" if i in human_check_todolist else "❌"} '+  str(output["scores"]))
        if i in human_check_todolist:
            if output["labels"][0] == "not to-do list memorize packing":
                if output["scores"][1] < 0.4:
                    dif_senc.append(f'{i} {text} ❌ ✅' + str(output["scores"]))
        else:
            if output["labels"][0] != "not to-do list memorize packing":
                dif_senc.append(f'{i} {text} ✅ ❌' + str(output["scores"]))
        
        # rank[output["labels"][0]].append(f'{i} {text}')
        # print(f'{i} {text} ✅{output["labels"]}'+  str(output["scores"]))
    
        # print(f"Time taken to classify sequence: {execution_time:.2f} seconds")

    print("NOt relate to travel: ",count_not_relate)
    print("Filtered: ",count_filtered, " / ", len(prompt_input)) 
    print("")
    
    for key in rank:
        print(key, len(rank[key]))
        for x in rank[key]:
            print(x)
    print("")
    print("Tổng hợp các câu lệch đáp án",len(dif_senc))
    for x in dif_senc:
        print(x)
    print("")
    print("Tổng hợp các câu liên quan đến du lịch " ,len(related_messages))
    for x in related_messages:
        print(x)
    return related_messages

msg = classify_sentence_todo(prompt)
    

Tổng hợp Tất cả câu phân loại
0 Hello mọi người tôi là Minh chủ chuyến đi này ❌ ❌ [0.6987407803535461, 0.30125921964645386]
1 Mọi người đều biết là đi Đà Nẵng rồi nhỉ ❌ ❌ [0.6661065816879272, 0.33389344811439514]
2 Chốt lịch là thứ 7 đến thứ 2 nhé mọi người. ✅ ❌ [0.5434091091156006, 0.456590861082077]
3 Hmm nghe có vẻ đưuọc đó ❌ ❌ [0.6805339455604553, 0.31946608424186707]
4 Sáng thứ 7 7h tập trung ở sân bay Tân Sơn Nhất nhé, đi chuyến sớm. ❌ ❌ [0.6912106275558472, 0.30878937244415283]
5 Sao sơm thế ? ❌ ❌ [0.7670605778694153, 0.2329394370317459]
6 Vì chuyến bay sớm, nên phải tập trung sớm để check-in. ✅ ❌ [0.5827493071556091, 0.4172506332397461]
7 Haiz lại phải dậy sớm rồi 😢 ❌ ❌ [0.8855754137039185, 0.11442457884550095]
8 Không sao, chuyến đi sẽ thú vị lắm đó. ❌ ❌ [0.948645293712616, 0.05135475471615791]
9 Còn lịch trình cụ thể thì mai bàn nhé ❌ ❌ [0.9970974922180176, 0.0029024670366197824]
11 Bàn luôn đi mai tôi phải học rồi ❌ ❌ [0.7314033508300781, 0.2685966491699219]
12 Bà này lắm ch

### Fetch api gemini đưa ra todolist

In [11]:
initialPrompt ='''
Hãy giúp tôi phân tích đoạn hội thoại và tạo một to-do list dưới dạng JSON string bỏ cái ngoặc ```json``` không cần cách hay xuống dòng. Chỉ lọc các hành động liên quan đến việc chuẩn bị đồ, tiền, giấy tờ (packing) hoặc đặt vé tàu xe, khách sạn(booking). 
Mỗi task có:
description: mô tả ngắn công việc 
category: tag ["packing","booking"]
member: Tên người phụ trách và id, nếu chung để trống
time: thời gian nếu có
priority: ["Cao", "Trung bình", "Thấp"]
Input:
Conversation:

Minh: Vì chuyến bay sớm, nên phải tập trung sớm để check-in.
Dũng: Thôi thôi tôi xin lại chiến tranh tg t3 bây h
Lan: Trưa tới Đà Nẵng đi ăn bánh tráng cuốn thịt heo nha, nghe bảo quán Trần ngon lắm.
Dũng: À ừ tôi cũng nghe nói về quán đó rồi, chắc phải thử thử hehehe =)).
Minh: Nhớ mang quần sịp =))
Minh: Để t mang kem chống nắng cho.
Lan: Nhớ đem theo máy ảnh để chụp ở Ngũ Hành Sơn!
Minh: Giữ vé máy bay cẩn thận khi lên máy bay!
Users: {"Minh": "id456","Dũng": "id123","Lan": "id879"}

Output:
{
"todolist":[
{"description":"Mang quần sịp","priority":"Trung bình","member":{"Dũng":"id123"},"category":"packing","time":""},
{"description":"Mang kem chống nắng","priority":"Trung bình","member":{"Minh":"id456"},"category":"packing","time":""},
{"description":"Đem máy ảnh để chụp hình ở Ngũ Hành Sơn","priority":"Trung bình","member":{"Lan":"id879"},"category":"packing","time":""},
]
}
'''

In [ ]:
import google.generativeai as genai
import json

from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

request = '''
Conversation:

Minh: Nhớ đến đúng giờ để không lỡ chuyến nha!
Dũng: Có ai mang theo ô không, nghe bảo thời tiết hơi thất thường.
Lan: Sáng mai đến Hội An ăn cao lầu không mọi người? Nghe nói quán Bà Bé ngon lắm.
Dũng: Chắc chắn phải thử, nghe là phải xếp hàng sớm nữa.
Minh: Ai có đồ bơi nhớ mang theo nhé, đến biển không có là phí lắm.
Minh: T mang thêm ít bánh mì để ăn nhẹ trên đường.
Lan: Nhớ đem sạc dự phòng nhé, cần cho cả ngày chụp ảnh đó!
Minh: Ai không có vé xe thì báo để mình đặt hộ luôn nhé.

Danh sách Users và id:
{
  "Minh": "id456",
  "Dũng": "id123",
  "Lan": "id879"
}
'''

# Sample related messages
data = {
    "prompt": request
}
chat =  model.start_chat(
    history=[
                {"role": "user","parts":  initialPrompt},
                {"role": "model","parts": "Tôi hiểu rồi, tôi sẽ nhận vào cuộc hội thoại, tổng hợp lại thành 1 json string có thể chuyển lại thành json danh sách công việc."}
            ]
  )

response = chat.send_message(data["prompt"])
parsed_response = json.loads(response.text)
formatted_json = json.dumps(parsed_response, indent=4, ensure_ascii=False)
print(formatted_json)  # Display in a well-formatted way



{
    "todolist": [
        {
            "description": "Mang theo ô",
            "priority": "Trung bình",
            "member": {
                "Dũng": "id123"
            },
            "category": "packing",
            "time": ""
        },
        {
            "description": "Mang đồ bơi",
            "priority": "Trung bình",
            "member": "",
            "category": "packing",
            "time": ""
        },
        {
            "description": "Mang bánh mì",
            "priority": "Trung bình",
            "member": {
                "Minh": "id456"
            },
            "category": "packing",
            "time": ""
        },
        {
            "description": "Mang sạc dự phòng",
            "priority": "Trung bình",
            "member": {
                "Lan": "id879"
            },
            "category": "packing",
            "time": ""
        },
        {
            "description": "Đặt vé xe",
            "priority": "Cao",
            "membe